In [1]:
import praw

In [2]:
reddit = praw.Reddit(client_id='redacted',
                    client_secret= 'redacted',
                    user_agent='redacted',
                    username='redacted',
                    password='redacted',)

In [4]:
end_of_season_2223 = 1685418710
beginning_of_season2223 = 1659744000
print(subs[0])

{'subName': 'LiverpoolFC', 'postTitle': 'FT', 'postFlair': 'Post Match'}


In [5]:
submission_list_liverpool = [
    submission for submission in reddit.subreddit('LiverpoolFC').search('FT') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post Match'
]
submission_list_arsenal = [
    submission for submission in reddit.subreddit('Gunners').search('Post Match') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post-Match Thread'

]

submission_list_manunited = [
    submission for submission in reddit.subreddit('RedDevils').search('Post Match') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223

]

submission_list_mancity = [
    submission for submission in reddit.subreddit('MCFC').search('Post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Match Day'

]

submission_list_spurs = [
    submission for submission in reddit.subreddit('coys').search('post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'PostMatch Thread'

]

submission_list_chelsea = [
    submission for submission in reddit.subreddit('chelseafc').search('post') if end_of_season_2223 >= submission.created_utc >= beginning_of_season2223 and submission.link_flair_text == 'Post-Match Thread'

]

In [6]:
import pandas as pd

def get_comment_data(end, beginning, submission_list):
    data = pd.DataFrame()

    for i in submission_list:
        submission = reddit.submission(i)
        finaltitle = i.title

        comment_data = []

        submission.comments.replace_more(limit=None)

        for comment in submission.comments.list():
            comment_data.append(str(comment.body))


        df = pd.DataFrame({finaltitle: comment_data})
        data = pd.concat([data, df], axis=1)
    return data


In [7]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [8]:
def get_mood_ratio(data):

    moodDF = pd.DataFrame()
    #for each column in the data
    for col in data.columns:
        col_list = []
        #for each comment in each column
        for comment in data[col].dropna():
            #split text into words
            temp = comment.split(' ')
            #find length
            length = len(temp)
            #need to restrict length of paragraph so that the sentiment pipeline can work
            if length > 100:
                new_comment = ' '.join(temp[:100])
                sent = sentiment_pipeline(new_comment)
                col_list.append(sent[0])

            else:
                sent = sentiment_pipeline(comment)
                col_list.append(sent[0])
        
        #
        moodDF.loc[:, col] = pd.Series(col_list)
        
        
    #cleaning moodDF, getting games out if there arent many comments
    new_moodDF = moodDF.copy()
    for column in new_moodDF.columns: 
        if new_moodDF[column].count() <= 7: 
            new_moodDF=new_moodDF.drop(column, axis=1) 
    
    #now making percentage
    percentage = pd.DataFrame()

    for col in new_moodDF.columns:
        num = 0
        num_negative = 0
        for item in new_moodDF[col].dropna():
            if item['label'] == 'NEGATIVE':
                num_negative += 1
            num += 1
        percentage[col] = [(num_negative/num)]
    
    return percentage


In [25]:
# liverpool_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_liverpool)
# liverpool_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\liverpool.csv')
#Above was done earlier and let run, but now I am accessing through the download

liverpool_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\liverpool.csv')
liverpool_data2223

,Unnamed: 0,FT Thread: Liverpool 7-0 Man United,FT Thread: Liverpool 2-5 Real Madrid,FT Thread: Palace 0-0 Liverpool,FT Thread - Liverpool 2-0 Wolves,FT Thread: Newcastle 0-2 Liverpool,FT Thread: Liverpool 2-0 Everton,FT Thread: Liverpool 0-0 Chelsea,FT Thread: Brighton 3-0 Liverpool,FT Thread: Brentford 3-1 Liverpool FC,...,FT Thread: AFC Ajax 0-3 Liverpool FC,FT Thread: Rangers FC 1-7 Liverpool FC,FT Thread - Liverpool 2-1 Ajax,FT Thread: Everton FC 0-0 Liverpool FC,FT Thread: United 2-1 Liverpool,FT Thread: Liverpool FC 2-1 Newcastle United FC,FT Thread: Liverpool FC 2-0 Rangers FC,FT Thread: Liverpool FC 1-1 Crystal Palace,FT Thread: Liverpool FC 9-0 AFC Bournemouth,FT Thread: Fulham FC 2-2 Liverpool FC
0,0,Wow! Sorry to be the one to say it but don’t t...,"Well that was shit, Real Madrid are definitely...",Well that was meh…. Something about gifs….\n\n...,"You know not to troll etc, and reply to this c...",Milner in the final minutes is everything.,Reminder that trolling rival subs carries a pe...,I miss Luis Diaz so bad,Gonna turn myself in to the police for watchin...,You know the drill; troll gets you perma banne...,...,"You know the drill, don't troll and report tro...",Reminder that trolling rival subs carries a pe...,Reminder that trolling rival subs carries a pe...,Reminder that trolling rival subs carries a pe...,"Well this isn’t ideal to say the least, rememb...",Liverpool vs Newcastle at Anfield is always gu...,Reminder that trolling rival subs carries a pe...,The sending off ironically woke us up. The dow...,Reminder that trolling rival subs carries a pe...,A lot of players who started should really be ...
1,1,Did not fucking expect that. We’re all in heav...,Wow who saw this coming from Gomez and Fabinho...,From 5 goals conceded to 0 😎. \n\n\n\nI hate e...,Nunez is just a joy to watch,Rest of PL: You beat Newcastle?\n\nLiverpool: ...,"Salah back to scoring ways, Nunez with an assi...",I fucking hate this fixture,That has to be the worst performance I have ev...,I aged 37 years in 90 minutes.,...,"Time to retire the cursed white swirly kit, lo...",I think we all needed that more that we’d care...,Collect 10 corners - receive 1 goal. Thank fuc...,Poor finishing. I'd like to see more of Firmin...,VVD and TAA very casual. Something is off.,Waste time -> concede in time added for your t...,Best Nunez has played for us IMHO. I know he d...,"Allison, Diaz, Elliot good. \n\nThat is all",Mo going to be so hungry midweek. You just kno...,Why does 36 y/o Milner want it more than half ...
2,2,Salah scoring and Man United fans leaving earl...,Completely deluded of me to even think we mig...,Pros:\n* we didn't lose\n* we kept a clean she...,4th clean sheet is a good foundation to build on.,"We got the 3 points, that’s all that matters....",DIVOCK ORIGI!!\n\nBest performance in a long t...,Literally our only attack right now is hoping ...,Don't worry guys. Bellingham will join in the ...,*sigh* I miss Jota and Diaz.,...,Let’s officially ban that white kit and make t...,False hope achieved we are beating City convin...,Massive result but even if this finished 1-1 t...,Milner got rinsed on several occasions today w...,Someone please resurrect jota,Fucking get in. \n\nCredit where its due thoug...,Darwin Nunez's movement and physicality is inc...,Spare a thought for Thiago who is going to hav...,Guys this was probably the first match I haven...,Fucking mitrović
3,3,The absolute best conversation on Sky commenta...,Allison: Lapse in concentration cost us the le...,Klopp just wanted Everton to go down,* Two points off Newcastle. 6 off Spud with ga...,Any time alisson sends Salah there’s like a 40...,How fast was Milner in the end 😅😅 blazing spee...,"Positives:\n\nClean sheet,\nBajectic,\nThe gam...",Klopp saying there was no sign of this coming ...,"Fully, completely deserved that loss",...,The endless Nunez debate on TV… he’s made seve...,Get fucked Rio you colossal wanker,Thiago is vital to this squad.,"Firmino looked really go

In [26]:
# arsenal_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_arsenal)
# arsenal_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\arsenal.csv')
#Above was done earlier and let run, but now I am accessing through the download
arsenal_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\arsenal.csv')
arsenal_data2223

,Unnamed: 0,Post Match Thread: Arsenal 3 - 2 AFC Bournemouth [English Premier League],Post Match Thread: Leicester City 0 - 1 Arsenal [English Premier League],Post Match Thread: Arsenal 4 - 0 Everton [English Premier League],Post Match Thread: Aston Villa 2 - 4 Arsenal [English Premier League],Post Match Thread: Arsenal 1 - 3 Manchester City [English Premier League],Post Match Thread: Sporting CP 2 - 2 Arsenal [UEFA Europa League],Post Match Thread: Arsenal 3 - 2 Manchester United [English Premier League],Post Match Thread: Everton 1 - 0 Arsenal [English Premier League],Post Match Thread: Arsenal 1 - 1 Brentford [English Premier League],...,Post Match Thread: Arsenal 5 - 0 Nottingham Forest [English Premier League],Post-Match Thread: Leeds United 0-1 Arsenal,Post Match Thread: Arsenal 2-1 AC Milan (ARS 4-3 penalties),Post Match Thread: Southampton 1 - 1 Arsenal [English Premier League],Post Match Thread: Arsenal 3 - 1 Tottenham Hotspur [English Premier League],Post Match Thread: PSV Eindhoven 2 - 0 Arsenal [UEFA Europa League],Post Match Thread: Arsenal 1 - 0 FC Zürich [UEFA Europa League],Post Match Thread: Arsenal 1 - 0 PSV Eindhoven [UEFA Europa League],Post Match Thread: Arsenal 1 - 3 Brighton & Hove Albion [English Carabao Cup],Post Match Thread: Arsenal 2 - 1 Fulham [English Premier League]
0,0,"THAT JUST HAPPENED!!!! What a comeback, one of...","2 goals in 2 for Martinelli, hopefully his con...",That 25-30 minute period around and after the ...,"Whatever his faults, Gabriel Magalhaes would g...",Never really felt like we were in it after the...,Saka playing 90 minutes in these matches will ...,This is why you sign a player like Trossard in...,"It was gonna happen eventually, but this was t...",Dropped 5 points in our 2 games. Points that s...,...,Arsenal fans: Edu can we have a mystery winger...,Ramsdale and his horn MOTM,Arsenal 3-0 Lyon(2-1 pens).\n\nLiverpool 1-3 L...,lyanco is a cunt,"I love this team, but week in... week out... M...",Totally outclassed. Well done PSV.\n\nNot muc...,Never in my life have I been so thrilled to ha...,Arsenal results in all comps since preseason \...,Well it's not like we lost to the PL 20th plac...,Imagine. You're a goal down and desperate but ...
1,1,Reiss has 5 goal contributions in 90 minutes o...,Final foul counter: 14 fouls from Leicester\n\...,"Front 4 all with goal scoring contributions, P...",JORGINHO THE ARSENAL LEGEND GET THE FUCK IN.\n...,I think if we had gotten out played and got sm...,Everyone looked gassed at the end. Lot of tire...,"Zinchenko easily MOTM, what a performance!",Went back and rewatched stoppage time. The bal...,I'm getting that old feeling watching us at th...,...,No suspensions for Jesus and Saliba might be t...,"I felt a great disturbance in the Force, as if...","Karl Hein is going to be some keeper huh, 20 a...",Fuck the ref. Jesus was abused out there,1. Dominant performance and a win.\n2. No inju...,"We simply weren’t good enough tonight, in term...",Something needs to be said about that fucking ...,*pops collar* \n \nThis years different,I do not Carabao this result.,lots of positives today: \n\n-\tthe fighting s...
2,2,OMFG WHAT EVEN IS THIS SEASON.\n\nIf the Gunne...,Big Gabi masterclass.,I can't understate how great it was to sit bac...,Jorginho not only paid for his transfer with t...,Remember when saliba scored an own goal and we...,"Considering the number of changes made, a draw...","Lee Dixon's commentary today was 10/10: ""I jus...",Annoyingly we get dyche ball twice where as ev...,Trossard and Tomiyasu should have started toda...,...,Ben White is pretty good at football,Getting 3 points off the single worst performa...,"Viera looks a lot better than before, much fit...",One of the Southampton Centre Backs has just b...,"Can’t judge our form yet, we still haven’t bea...",That was hardly worth watching. It's not a big...,Fuck everything about that game. And a special...,Shots 25-4. Got nervous we'd get FM'd in the e...,Kinda don’t care. Feels

In [27]:
# manunited_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_manunited)
# manunited_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\manunited.csv')
#Above was done earlier and let run, but now I am accessing through the download
manunited_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\manunited.csv')
manunited_data2223

,Unnamed: 0,[Post Match Thread] Liverpool 7 - 0 Manchester United,[Post Match Thread] Manchester United 2 - 1 Barcelona [United win 4 - 3 on Aggregate],[Post Match Thread] Manchester United 3 - 1 West Ham United,"[Post Match Interview] Ten Hag: ""You can lose a game but not in this way"" ""The second half is unprofessional, it can't happen. We have to talk about it.""","[Post-Match Quotes] Roy Keane: ""If you lose 7-0 you’d go into hiding, never mind for a few days. I think I’d go missing for a few months it really is that embarrassing for the players."" “Thank god I've never been part of a team that's been beaten by that much playing for Man United.”",[Post Match Thread] Barcelona 2 - 2 Manchester United,[Post Match Thread] Manchester United 3 - 0 Leicester City,[Post-Match Thread] Leeds United 0 - 2 Manchester United,[Post Match Image/Meme/Appreciation Thread] Manchester United 2 - 1 Barcelona,...,[Post Match Thread] Manchester United 1 - 2 Brighton & Hove Albion,"[Post Match Quotes] Ten Hag: ""The result is clear, it never lies. In football you always get your own earnings. Today we were not good enough."" #MUFC",[Post Match Thread] Brentford 4 – 0 Manchester United,Erik ten Hag FULL post-match press conference | Manchester United 3-1 Reading,Erik ten Hag post-match interview with NBC Sport,[Post Match Thread] Southampton 0 - 1 Manchester United,Luke Shaw post match interview with BT sport,"Erik, Bruno & Marcus React To A Big 3 Points Post-Match Reaction",Erik ten Hag EMBARGOED post-match press conference Manchester United 3-1 Everton,Erik ten Hag FULL post-match press conference Manchester United 3-0 Bournemouth
0,0,"Hey, fellas. Just a reminder to please report ...",Today I feel Brazilian.,This game reminded me of the SAF days more tha...,Any update on Case? He's miles behind in pace ...,Ten Hag on if the players explained the capitu...,The fact that we just drew 2-2 at Camp Nou and...,Luke Shaw is an elite footballer. Everyone kne...,Martinez coming in changed the game too,https://i.imgur.com/P3UtFTP.jpg,...,"We've lost: Pogba, Matic, Mata, Lingard in mid...",We've played a shit tonne of games and we can'...,Title is wrong should be **[Post-Mortem Thread]**,The right guy for the job,Thanks for sharing. They got 10x more insight ...,Defence was MASSIVE yet again,"He’s always honest in his interviews, I like w...","Wow, Rashford expressed unhappiness about the ...",I get the feeling that if someone was talking ...,"I keep watching matches and thinking, “we’ve g..."
1,1,Never have i ever expected this backline with ...,That fucking block by Varane.,Moyes and losing at Old Trafford name a better...,"**More Quotes :**\n\nTen Hag: ""It was not abou...",What really grinds my gears now that I've had ...,Gavi will be out. Pedri will be out. Busquets ...,Second half was pure sex.,Ten Hag's tactics at the 60 minutes mark have ...,[How the conversation at dinner during the wee...,...,"It's going to be a long season, boys.",I'll be honest I did think we will draw this g...,Someone tell the glazers that you don't get as...,"Giving me uncle Evra's ""i love this game"" vibe...",I also loved how he called out that no one wan...,Let that be a lesson to all: United DO NOT los...,links to reddit homepage for me,NaN,Who is the first journalist? What an unbelieva...,He’s so business-like winning feels like the e...
2,2,Banter when the United app asks for our man of...,Licha v Lewandowski one of the most one sided ...,Weghorst 👏👏\n\nWhat a fucking shift from the b...,Very true and it's not the first time it's hap...,"When our rivals were dogshit, we could hardly ...",Just so you're all aware: we have outplayed th...,We need to keep Lindelof and sell Maguire in t...,I fucking loved how the Leeds atmosphere just ...,https://twitter.com/LONEXBA/status/16288753764...,...,I loved football between June and July.,Let's be real. You can't contest top 4 with ju...,Ivan Toney: “We know they’ve got some great pl...,I don't complain and it's not Copacaban

In [28]:
# mancity_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_mancity)
# mancity_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\mancity.csv')
#Above was done earlier and let run, but now I am accessing through the download
mancity_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\mancity.csv')
mancity_data2223

,Unnamed: 0,[Post Match Thread] RB Leipzig 1 - 1 Manchester City,[Post Match Thread] Manchester City 2 - 0 Newcastle,[Post Match Thread] Arsenal 1 - 3 Manchester City,[Post Match Thread] Bournemouth 1 - 4 Manchester City,[Post Match Thread] Bristol City 0 - 3 Manchester City,[Post Match Thread] Tottenham 1 - 0 Manchester City,[Post Match Thread] Manchester United 2 - 1 Manchester City,[Post Match Thread] Manchester City 1 - 0 Arsenal,[Post Match Thread] Southampton 2 - 0 Manchester City,...,[Post Match Thread] Manchester City 2 - 1 Borussia Dortmund,[Post Match Thread] Manchester City 4 - 0 Southampton,[Post Match Thread] Manchester City 5 - 0 FC Copenhagen,[Post Match Thread] Manchester City 4 - 2 Crystal Palace,[Post Match Thread] Aston Villa 1 - 1 Manchester City,[Post Match Thread] Wolves 0 - 3 Manchester City,[Post Match Thread] Manchester City 6 - 0 Nottingham Forest,[Post Match Thread] Sevilla 0 - 4 Manchester City,[Post Match Thread] West Ham 0 - 2 Manchester City,[Post Match Thread] Manchester City W 1 - 1 Manchester United W
0,0,With the cost of living crisis not abating any...,With the cost of living crisis not abating any...,Trolling bad. Trolling = bans. No Trolling.,With the cost of living crisis not abating any...,With the cost of living crisis not abating any...,With the cost of living crisis not abating any...,With the cost of living crisis not abating any...,With the cost of living crisis not abating any...,With the cost of living crisis not abating any...,...,Things learned from this match\nNever play Gre...,Akanji is so good man. Another txiki masterclass,My heart hurts so much for Grealish dropping t...,"Hi all, feel free to put in your ratings for t...","Hi all, feel free to put in your ratings for t...",Grealish is good and I will die on that hill,"Hi all, feel free to put in your ratings for t...","Hi all, feel free to put in your ratings for t...",Julian pretty much knocking out Zouma with a b...,"Whats strange to me, no correct me if I am wro..."
1,1,I really like this 'Never pass to Haaland' tac...,How can you not love Haaland?\n\n* Scores so m...,There were some great performances out there b...,I saw this floating around. Make me smile\n\nh...,Ake stepping up further into the attack was a ...,What ever pep saw in that second preseason sin...,Absolutely fucking ridiculous. We were nowhere...,Shut those fucking commentators up. Not an oun...,To think we've knocked Liverpool & Chelsea out...,...,"John Stones MOTM, scored a screamer, tried to ...",We’re due a Der Speigel article,Haaland running back on to the pitch to clap o...,Fantastic game. Despite Haaland's great hatric...,Annoying but not the end of the world by any m...,I can't stress enough how good akanji has been...,The Haaland-Nunez debate ended before Nunez co...,Biggest disappointment was the lack of Navas c...,Haaland called his game “a bit shit” because h...,great second half from us and glad to see all ...
2,2,Why buy Haaland when we are just making him lo...,https://preview.redd.it/btgzitosyrla1.jpeg?wid...,Dias has to start the rest of the way. Incredi...,I know Alvarez and foden don't provide that mu...,"KDB finally scoring, Foden back in form, Rodri...",Bad performances all around. Grealish and Ake ...,Grealish deserved better. So happy for him. Wo...,If today was a league match we’d be saying we ...,The result is what it is. The performances fro...,...,Pep hugging him then getting right back to arg...,What a signing Akanji is proving to be. Cancel...,Respect to their Keeper who singlehandedly kep...,Bernardo was the catalyst in the comeback. The...,At least Newcastle played well and got their d...,"haaland was absolutely furious in the end, bec...",Haaland rightfully will get the plaudits but A...,Akanji and Gomez played well today.,"So so glad we've got a new contract for Rodri,...",Manchester United L\*
3,3,So sick and tired of Haaland being ignored. We...,It’s nice to win a game and not have the oppos...,Haaland celebrating like 

In [29]:
# spurs_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_spurs)
# spurs_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\spurs.csv')
#Above was done earlier and let run, but now I am accessing through the download
spurs_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\spurs.csv')
spurs_data2223

,Unnamed: 0,Post-Match Thread: Wolves vs. Spurs (4 Mar 2023),Post-Match Thread: Sheffield United vs. Spurs (1 Mar 2023),Post-Match Thread: Spurs vs. Chelsea (26 Feb 2023),Post-Match Thread: Spurs vs. West Ham (19 Feb 2023),Post-Match Thread: Leicester City vs. Spurs (11 Feb 2023),Post-Match Thread: Man City vs. Spurs (19 Jan 2023),Post-Match Thread: Spurs vs. Arsenal (15 Jan 2023),Post-Match Thread: Spurs vs. Aston Villa,Post-Match Thread: Fulham vs. Spurs (23 Jan 2023),...,[Post Match Thread] Brighton Vs. Tottenham Hotspur (PL 08/10/22),[Post-Match Thread] Spurs [6] - 2 Leicester,[Post Match] Frankfurt Vs. Tottenham Hotspur (CL 04/10/22),Post-Match Thread: Chelsea vs. Spurs (14 Aug 2022),Post-Match Thread: Sporting Lisbon vs. Spurs (13 Sep 2022),Post-Match Thread: West Ham vs. Spurs (31 Aug 2022),Post-Match Thread: Spurs vs. Olympique de Marseille (7 Sep 2022),Post-Match Thread: Nottingham Forest vs. Spurs (28 Aug 2022),Post-Match Thread: Spurs vs. Wolves (20 Aug 2022),Post-Match Thread: Spurs vs. Fulham (3 Sep 2022)
0,0,Someone should be fired for those substitution...,Couldn’t do what Wrexham could four times,My grandad died this morning aged 94. Was Tott...,This was perfect example of Romero getting the...,What can you even say really. Absurd performan...,Reminder: we're celebrating our subreddit with...,Report the Gooners coming in. They'll be banne...,"\nYeah, I’ve got nothing to defend this team t...",Reminder: we're celebrating our subreddit with...,...,Don’t underestimate this win. Brighton are fly...,We need much much more of the 3-man midfield w...,One of the most frustrating games I’ve watched...,Tuchel grabbed Conte’s hand and pulled him? Th...,Such a shame there was no football on this week,I think Perisic and Son need to work on their ...,Richarlison shooting up the power rankings of ...,Everton fans were right. I fucking love him.,"Perisic is different gravy, what a player",Spurs should have won that 5-0 instead i’m ove...
1,1,Are we living in a Arsenal fan-fic?,Not losing to Marine is our biggest FA Cup ach...,Ben Davies is our best LWB,Chelsea downfall\n\nWest Ham relegation zone\n...,"Welcome to Tottenham Pedro Porro, you’ve now s...",How about those 3 minutes at the end of the fi...,"Another big match, another match where a playe...",the only positive thing from this game is that...,Romero's ability to leave his position and fin...,...,For Gian Piero,"Cheers Son, I’m crying.",the only thing i’m going to choose to remember...,Competitive games between us and Chelsea are t...,Conte talking about big squad then only uses 1...,Didn't have much of an attack to win this matc...,Ben Davies run made the Richarlison goal,Another Sanchez clean sheet,First half wasn't great but we made a shitton ...,Richi and Son not getting their goals hurts my...
2,2,Utterly inexcusable that our staff see wolves ...,That is the worst last 8 I remember seeing in ...,Romero played like a man possessed today. Love...,"""I came here to sit on the bench and hurt West...",I am going to the gym,"For a defensive side, we sure do concede a lot",Don’t worry guys we’re winning the Champions l...,Think we’ll need more than $30 mil,Our fans were so fucking loud tonight,...,That Piero shirt Hugo holding 😢,[Son rn](https://m.youtube.com/watch?v=Y45PQG1...,Tottenham try to make a quality final third pa...,I already love Bissouma,Hopefully this will be the wake up call,Embarrassing effort second half,Really fucking happy for Richarlison! To start...,Stat of the day: We haven't conceded a goal in...,[deleted],I enjoyed Lenglet
3,3,That is 1000% on the managers. Was a very winn...,Running out of adjectives to describe this fee...,The players fought for the badge today. Wouldn...,"When I envisioned the perfect wingback system,...",Let’s just hope Bentancur is okay.,"We've played Chelsea, Man U, Newcastle, Liverp...","Lloris, you've been a great servant, but it's ...",Midtable football gets you... Midtable \n\n6 w...,Emerson actually had a good game and

In [30]:
# chelsea_data2223 = get_comment_data(end_of_season_2223, beginning_of_season2223, submission_list_chelsea)
# chelsea_data2223.to_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\chelsea.csv')
#Above was done earlier and let run, but now I am accessing through the download
chelsea_data2223 = pd.read_csv(r'C:\Users\mjjyo\OneDrive\Desktop\soccermooddata\chelsea.csv')
chelsea_data2223

,Unnamed: 0,[Post-Match thread] Tottenham 2-0 Chelsea - Premier League (Matchday 25),Post Match: Chelsea 2 : 0 Borussia Dortmund [Champions League],[Post-Match thread] Chelsea 1-0 Leeds - Premier League (Matchday 26),[Post-Match thread] Chelsea 0-1 Southampton - Premier League (Matchday 24),[Post-Match thread] Borussia Dortmund 1-0 Chelsea - UEFA Champions League (Round of 16),[Post-Match thread] Chelsea 0-0 Fulham - Premier League (Matchday 22),[Post Match Thread] West Ham 1-1 Chelsea | EPL,[Post-Match thread] Liverpool 0-0 Chelsea - Premier League (Matchday 21),[Post Match Thread] Fulham vs Chelsea - EPL,...,[Post-Match thread] Chelsea 1-1 Red Bull Salzburg - UEFA Champions League (Group E - 2),[Post-Match thread] Chelsea 3-0 AC Milan - UEFA Champions League (Group E - 3),[Post-Match thread] AC Milan 0-2 Chelsea - UEFA Champions League (Group E - 4),Post-Match Thread: West Ham 1 - 1 Chelsea [Premier League],[Post-Match thread] Southampton 2-1 Chelsea - Premier League (Matchday 5),[Post-Match thread] Chelsea 2-1 West Ham - Premier League (Matchday 6),[Post-Match thread] Aston Villa 0-2 Chelsea - Premier League (Matchday 11),[Post-Match thread] Chelsea 2-1 Leicester - Premier League-Matchday 4,[Post-Match thread] Chelsea vs Tottenham ( Matchday 2 ),POST MATCH THREAD : LEEDS VS CHELSEA
0,0,I think 38 points should make us safe right? R...,Potter in am I right guys?,A friendly reminder to report any comments tha...,A friendly reminder to report any comments tha...,Let your frustrations out by filling out our c...,A friendly reminder to report any comments tha...,"Apologies for the delay, Reddit as a whole cou...",A friendly reminder to report any comments tha...,A friendly reminder to report any comments tha...,...,A friendly reminder to report any comments tha...,A friendly reminder to report any comments tha...,Reece was walking on the field at the end of g...,Potter on the hand ball - “I haven’t gone back...,Link to post match survey\n\nhttps://www.reddi...,A friendly reminder to report any comments tha...,A friendly reminder to report any comments tha...,Post match survey: https://www.reddit.com/r/ch...,"Anthony Taylor may be an awful referee, but de...",[Link to post-match survey](https://www.reddit...
1,1,this was probably the most painful performance...,KAAI HAVERTZ IS THE BEST ON EARTH,We won a game 🥲,We lost to the bottom team of Premier League. ...,At least Arsenal lost,I was actually excited for this match. Looking...,Cucurella has one play and it’s dribble forwar...,Badiashile looks good,We.fucking.suck.,...,The Broja counter attack following that corner...,Man did we need that. Great response after a s...,Ordinarily a player walking off the pitch woul...,Not sure how we’re so uncreative with that lin...,What a shitshow.,Three things: \n\n- Start Chilwell\n\n- Mendy ...,"Happy with potter, saw the problems, quickly m...",Great fucking response from going down to 10 m...,Pathetic from Taylor and the VAR ref,we were looking so good with the intensity and...
2,2,"Zero plan, no patterns of play or distinct sty...","Unironically, the lads gave it everything toni...",Hey we won a game! All the new players that ca...,Disgusting. Boring. Waste of time. Fuck off.,"Someone please, for the love of god, just put ...",That fulham defense really bossed the game tod...,Why doesn’t Cucurella ever pass the damn ball ...,Mudryk is that dude,Felix looked like the player we hoped he would...,...,"Seems like Ziyech wasn't sabotaging Tuchel, he...",Genuinely feels like this game might have kick...,"Nice game, enjoyable evening.\n\nProps to moun...","In all my time watching this season, I don't t...",Sterling must hate this attack with a passion,CHILLY HIVE WE BACK UP\n\nFofana faultless. Am...,"Kepa MOTM for me, what a showing today. Ganna ...",Thiago fucking Silva. I will never get tired o...,Gave up 2 goals because of 2 refereeing mistak...,"3 games in, still zero goal from the front 3. ..."
3,3,The cheek to say “I’ve just come out a cr